### Context Caching Gemini 
- Gemini API offers context caching feature 
- Reduces costs by caching input tokens for repeated requests in AI workflows 
- Default TTL of 1 hour 
- Available for Gemini 1.5 Pro and Flash versions 
- Recommended for scenarios with substantial initial context referenced by shorter requests 
- Billing based on cached token count and storage duration 
- Refer to Gemini API pricing page for more details

In [1]:
!pip install -qU 'google-generativeai>=0.7.0'


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import google.generativeai as genai
from google.generativeai import caching

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
genai.configure(api_key="<KEY>")

In [7]:
!wget https://raw.githubusercontent.com/kunjee17/mahabharata/master/books/01.txt

--2024-07-05 05:07:14--  https://raw.githubusercontent.com/kunjee17/mahabharata/master/books/01.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1345896 (1.3M) [text/plain]
Saving to: ‘01.txt’

01.txt              100%[===================>]   1.28M  --.-KB/s    in 0.04s   

2024-07-05 05:07:15 (30.8 MB/s) - ‘01.txt’ saved [1345896/1345896]



In [8]:
document = genai.upload_file(path="./01.txt")

In [11]:
!head 01.txt

BOOK 1

ADI PARVA

Translated into English Prose from the Original Sanskrit Text

by

Kisari Mohan Ganguli



In [9]:
model_name = "gemini-1.5-flash-001"

maha_cache = caching.CachedContent.create(
    model=model_name,
    system_instruction="You are an expert transcript analyzer, you can understand and summarize larger texts",
    contents=[document],
)

maha_cache

CachedContent(
    name='cachedContents/69ckigsl6i7c',
    model='models/gemini-1.5-flash-001',
    display_name='',
    usage_metadata={
        'total_token_count': 330894,
    },
    create_time=2024-07-05 05:08:04.291582+00:00,
    update_time=2024-07-05 05:08:04.291582+00:00,
    expire_time=2024-07-05 06:08:03.867270+00:00
)

In [12]:
import datetime

maha_cache.update(ttl=datetime.timedelta(hours=4))
maha_cache

CachedContent(
    name='cachedContents/69ckigsl6i7c',
    model='models/gemini-1.5-flash-001',
    display_name='',
    usage_metadata={
        'total_token_count': 330894,
    },
    create_time=2024-07-05 05:08:04.291582+00:00,
    update_time=2024-07-05 05:09:23.950301+00:00,
    expire_time=2024-07-05 09:09:23.928522+00:00
)

In [13]:
maha_model = genai.GenerativeModel.from_cached_content(cached_content=maha_cache)

response = maha_model.generate_content("give me the summary of chatper 1")
print(response.text)

Chapter 1 of the Adi Parva of the Mahabharata introduces the story of the epic through the character of Sauti, a sage who narrates the epic to a group of other sages at a great sacrifice.  Here's a summary of the chapter:

* **Sauti arrives at the sacrifice:** Sauti, a learned sage who has traveled extensively and heard the epic from Vaisampayana, arrives at the great sacrifice organized by the sage Saunaka. He tells the other sages he is eager to share the epic they are requesting. 
* **The Importance of the Mahabharata:** The sages express their desire to hear the epic, specifically the Bharata portion, highlighting its importance as a sacred work that can dispel fear and provide knowledge.
* **The Creation Story and the Purpose of the Mahabharata:** Sauti begins his narration with a creation myth, describing how the universe was born from a cosmic egg and how the Devas, humans, and other beings came into existence. He then emphasizes that the Mahabharata, composed by the sage Vyasa,

In [14]:
response.usage_metadata

prompt_token_count: 330904
cached_content_token_count: 330894
candidates_token_count: 443
total_token_count: 331347

In [15]:
chat = maha_model.start_chat()
response = chat.send_message("What happened at the end of chapter 1?")
print(response.text)

At the end of Chapter 1, Dhritarashtra, overwhelmed with grief for the loss of his sons in the Kurukshetra war, expresses his desire to end his life.  Sanjaya, his charioteer, consoles him by reminding him of the many powerful and virtuous kings of the past who also met their end, and that grief over inevitable fate is foolish. 

Dhritarashtra, comforted by Sanjaya, gives up his suicidal thoughts. 



In [16]:
response.usage_metadata

prompt_token_count: 330905
cached_content_token_count: 330894
candidates_token_count: 93
total_token_count: 330998

In [17]:
maha_model.count_tokens("How many kings are involved in this chapter?")


total_tokens: 9